# Imports & Constants

In [19]:
from __future__ import division
import numpy as np
import sympy
sympy.init_printing()
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact
from scipy.integrate import odeint
from scipy.optimize import fsolve

In [20]:
g = 9.807
P_atm = 82 #kPa

# Fluid properties

In [21]:
ρ = 1000 #kg/m^3
SG = ρ/1000
μ = 8.9E-4 #Pa.s
P_sat = 0.66435 #kPa

# Pipeline properties

In [22]:
d = 110.08 #in mm
d_inch = d * 0.03937 #in inches
D = d/1000 #in m
epsilon = 0.015 #in mm
W_n = 34000/60/60 #kg/s
W_d = 1.15*W_n #kg/s

# Suction line

In [23]:
length_suction = 2
dZ_suction = -(1 + 0.8*5)
resist_suction = [4, 0, 0, 2, 1, 0, 1, 0]
# (90 deg elbows, large radius elbow, Butterfly valves, Gate Valves, soft tee, hard tee, 
# Pipe enterances, Pipe exits)

# Delivery line

In [24]:
length_delivery = 14
dZ_delivery = 6
resist_delivery = [8, 0, 0, 6, 7, 0, 0, 1]
# (90 deg elbows, large radius elbow, Butterfly valves, Gate&Ball Valves, soft tee, hard tee, 
# Pipe enterances, Pipe exits)

# Control valve characteristics

In [25]:
CV_size = 110.08
# in mm
C_cv = 124
# single port Ccv values
fx_d = 1 
fx_n = 0.6

In [26]:
reducer_θ = 30
enlarger_θ = 30
if CV_size < d:
    β = CV_size/d
    CV_smaller = True
elif CV_size > d:
    β = d/CV_size
    CV_smaller = False
else:
    CV_smaller = True 
# if the valve is the same size as the line β = 1 and the K equations become 0

if CV_smaller == True:
    # if the control valve is smaller than the normal line the K values must be adjusted for the larger line since formula are
    # given in terms of the flow in the smaller line
    if reducer_θ < 45:
        reducer_K = 0.81*np.sin(reducer_θ/2)*(1-β**2) * β**4
    else:
        reducer_K = 0.5 (1-β**2)*(np.sin(reducer_θ/2))**0.5 * β**4
    if enlarger_θ < 45:
        enlarger_K = 2.61*np.sin(enlarger_θ/2)*(1-β**2) * β**4
    else:
        enlarger_K = (1-β**2)**2 * β**4
elif CV_smaller == False:
    if reducer_θ < 45:
        reducer_K = 0.81*np.sin(reducer_θ/2)*(1-β**2)
    else:
        reducer_K = 0.5 (1-β**2)*(np.sin(reducer_θ/2))**0.5
    if enlarger_θ < 45:
        enlarger_K = 2.61*np.sin(enlarger_θ/2)*(1-β**2)
    else:
        enlarger_K = (1-β**2)**2

# Functions

In [27]:
def reynolds(u):
    Re = (D * u * ρ) / μ
    return Re

def dPelevation(dh):
    dPel = ρ * g * dh / 1000
    return dPel

def dPeq_f(W):
    dPeqr = 33.44
    Wr = 34000/60/60
    dPeq = dPeqr * (W/Wr)**2
    return dPeq

def dPcv_f(W, fx):
    V = W/ρ
    V_gal = V*264.172*60
    dPcv = (V_gal/(C_cv*fx))**2 * SG * 6.89476
    return dPcv

def dP_friction(Lp, u, resistances):
    # Resistances should be given as
    # (90 deg elbows, large radius elbow, Butterfly valves, Gate Valves, Pipe enterances, Pipe exits)
    # Valves are taken as fully open
    # All connections are taken as flanged
    K1 = [800, 800, 800, 300, 160, 0]
    Kinf = [0.25, 0.2, 0.25, 0.1, 0.5, 1]
    # Initial Guess for f_prime
    sqrt_f_prime = 0.1414
    Re = reynolds(u)
    # Iterate to find f_prime using colebrook
    difference = 1
    while abs(difference) > 1E-5:
        sqrt_f_prime = 1/(-2 * np.log10(epsilon/D/3.7 + 2.51/(Re*sqrt_f_prime)))
        difference = 1/(sqrt_f_prime) - (-2 * np.log10(epsilon/D/3.7 + 2.51/(Re*sqrt_f_prime)))
    f_prime = sqrt_f_prime**2
    # Calculate the resistance coefficient for resistances
    Ktot = 0
    for i in range(0, 6, 1):
        if i < 4:
            K = K1[i]/Re + Kinf[i]*(1 + 1/d_inch)
            Ktot += K
        else:
            K = K1[i]/Re + Kinf[i]
            Ktot += K
    # Add resistance for the enlarger & reducer
    Ktot += enlarger_K
    Ktot += reducer_K
    # Add Resistance for the pipe
    K_pipe = f_prime*Lp/D
    Ktot += K_pipe
    # Calculate friction loss
    dP_friction = Ktot*(ρ*u**2/2000)
    return dP_friction

def suction(W):
    # Mass flow rate in kg/s
    u = (W/ρ) / ((np.pi*D**2)/4)
    dPel = dPelevation(dZ_suction)
    dPke = 1*ρ/2000*(u**2)
    dPf = dP_friction(length_suction, u, resist_suction) #Excludes control valve
    P_suction_flange = -(dPel + dPke + dPf)
    return P_suction_flange

def delivery(W, fx_cv):
    # Mass flow rate in kg/s
    u = (W/ρ) / ((np.pi*D**2)/4)
    dPel = dPelevation(dZ_delivery)
    dPke = 1*ρ/2000*(-u**2)
    dPf = dP_friction(length_delivery, u, resist_delivery) #Excludes control valve
    dPcv = dPcv_f(W, fx_cv)
    print (dPcv)
    dPeq = dPeq_f(W)
    P_delivery_flange = dPel + dPke + dPf + dPeq + dPcv
    return P_delivery_flange

# Calculations
## Design conditions

In [28]:
P_delivery_flange = delivery(W_d, fx_d) + P_atm
P_suction_flange = suction(W_d) + P_atm

dPa = P_delivery_flange-P_suction_flange

NPSH_a = (P_suction_flange - P_sat)/(g*SG)
print("The pressure at the delivery flange is ", str(P_delivery_flange), "kPa abs")
print("The pressure at the suction flange is ", str(P_suction_flange), "kPa abs")
print("The pressure supplied by the pump is ", str(dPa), "kPa")
print("NPSHa is ", str(NPSH_a), "m fluid")

13.289246753700507
The pressure at the delivery flange is  209.467038072 kPa abs
The pressure at the suction flange is  127.260824794 kPa abs
The pressure supplied by the pump is  82.2062132778 kPa
NPSHa is  12.9087870698 m fluid


## Nominal Conditions

In [29]:
P_delivery_flange = delivery(W_n, fx_n) + P_atm
P_suction_flange = suction(W_n) + P_atm

dPa = P_delivery_flange-P_suction_flange

NPSH_a = (P_suction_flange - P_sat)/(g*SG)
print("The pressure at the delivery flange is ", str(P_delivery_flange), "kPa abs")
print("The pressure at the suction flange is ", str(P_suction_flange), "kPa abs")
print("The pressure supplied by the pump is ", str(dPa), "kPa")
print("NPSHa is ", str(NPSH_a), "m fluid")

27.912721599875045
The pressure at the delivery flange is  210.598982165 kPa abs
The pressure at the suction flange is  128.179548252 kPa abs
The pressure supplied by the pump is  82.4194339139 kPa
NPSHa is  13.0024674469 m fluid


In [31]:
W_n